In [1]:
path_to_home = "../"
#path_to_home = "./drive/MyDrive/receiptlayoutlm/"

In [2]:
from huggingface_hub import Repository
hf_repo = Repository(path_to_home + "hugging_face_repo")

In [ ]:
from transformers import LayoutLMv2FeatureExtractor, LayoutXLMTokenizer, LayoutXLMProcessor
from transformers import LayoutLMv2ForTokenClassification
from transformers import pipeline
from transformers import AutoConfig

feature_extractor_xlm = LayoutLMv2FeatureExtractor()
tokenizer_xlm = LayoutXLMTokenizer.from_pretrained("microsoft/layoutxlm-base")
processor_xlm = LayoutXLMProcessor(feature_extractor_xlm, tokenizer_xlm)
model_xlm = LayoutLMv2ForTokenClassification.from_pretrained("sibrun/receiptlayoutlm", use_auth_token=True)

In [ ]:
config_xlm = AutoConfig.from_pretrained("sibrun/receiptlayoutlm", use_auth_token=True)
model_infer = pipeline('ner', model=model_xlm, config=config_xlm, tokenizer=tokenizer_xlm, feature_extractor=feature_extractor_xlm)

In [ ]:
model_infer.push_to_hub("sibrun/receiptlayoutlm_ner", commit_message='final ner pipline')

In [ ]:
label_names = ['company', 'date', 'address', 'total']
labels = ['O'] + label_names
num_labels = len(labels)
ids_to_labels = {k: v for k, v in enumerate(labels)}
labels_to_ids = {v: k for k, v in enumerate(labels)}

In [8]:
from PIL import Image
import numpy as np
import torch

path_to_image = "../data/X00016469612.jpg"
image = Image.open(path_to_image)
features = feature_extractor_xlm(image, return_tensors="pt")
tokenizer_output = tokenizer_xlm(
        text=features['words'],
        boxes=features['boxes'],
        is_split_into_words=True,
        truncation=True,
        return_tensors='pt')
model_input = tokenizer_output
model_input['image'] = features['pixel_values']
with torch.no_grad():
        model_output = model_xlm(**model_input)
predictions = np.argmax(model_output.logits.tolist(), axis=-1)
tokens = [tokenizer_xlm.convert_ids_to_tokens(ids) for ids in tokenizer_output['input_ids'].tolist()]
ner_labels = [[ids_to_labels[p] for p in prediction] for prediction in predictions]
token_label_list = zip(tokens, labels)

/Users/simon/PycharmProjects/receiptlayoutlm/venv/lib/python3.9/site-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:772: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  torch.arange(
/Users/simon/PycharmProjects/receiptlayoutlm/venv/lib/python3.9/site-packages/transformers/models/layoutlmv2/modeling_layoutlmv2.py:782: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, us